## Better Way 87
### 호출자를 API로부터 보호하기 위해 최상위 Exception을 정의하라

### 모듈 API에서 중요한 부분

* 모듈 내에 정의한 함수/클래스
* 모듈 내에 정의한 Exception들 

### 아래 두 가지 중 어느 것을 써도 상황은 모두 비슷하다.
* 파이썬 언어와 표준 라이브러리에 내장된 예외 계층구조
* 직접 정의한 새로운 예외타입



In [1]:
# 직접 만든 모듈의 함수에 잘못된 파라미터 전달되면 ValueError 발생 

# my_module.py
def determine_weight(volume, density):
    if density <= 0:
        raise ValueError('밀도는 0보다 커야 합니다.')

이대로 ValueError를 쓰는 게 더 타당할 수 도 있지만... 
### API의 경우 새로운 예외 계층 구조를 정의하는 편이 더 강력하다.

### API에서 발생하는 예외의 계층구조
1. 모듈에 최상위 Exception 정의
2. 모듈이 발생시키는 다른 모든 예가 1.의 최상위 예외를 상속하게 만듦

In [2]:
# API의 새로운 예외계층 구조

# my_moudle.py
class Error(Exception):
    """이 모듈에서 발생할 모든 예외의 상위 클래스"""

class InvalidDensityError(Error):
    """밀도 값이 잘못된 경우"""

class InvalidVolumeError(Error):
    """부피 값이 잘못된 경우"""
    
def determine_weight(volume, density):
    if density < 0:   # <= 아닌 것은 저자의 의도적인 버그
        raise InvalidDensityError('밀도는 0보다 커야 합니다.')
    if volume < 0:    # <= 아닌 것은 저자의 의도적인 버그
        raise InvalidVolumeError('부피는 0보다 커야 합니다.')
    if volume == 0:   
        density / volume   # density를 0으로 나누는 것은 저자의 의도적인 버그

### 최상위 예외가 있으면 API사용자들이 이 모듈 안에서 발생한 모든 오류를 더 쉽게 찾을 수 있다,

In [3]:
import my_module
import logging

# 함수 호출시 try/except 문을 사용하여 최상위 예외 발생

try:
    weight = my_module.determine_weight(1,-1)
except my_module.Error:
    logging.exception('예상치 못한 오류')

ERROR:root:예상치 못한 오류
Traceback (most recent call last):
  File "<ipython-input-3-09f9f8a369f4>", line 7, in <module>
    weight = my_module.determine_weight(1,-1)
  File "C:\Users\PC\Documents\GitHub\into_the_python1\AIFFEL\Python Better Ways\my_module.py", line 31, in determine_weight
    raise NegativeDensityError('밀도는 0보다 커야 합니다.')
my_module.NegativeDensityError: 밀도는 0보다 커야 합니다.


### 기대효과: 우리가 제공하는 API를 호출하는 코드를 보호할 수 있다.
#### 1. logging.exception 함수가 있어서 더 쉽게 디버깅 할 수 있다.
함수가 잡아낸 예외의 전체 스택 트레이스를 출력하니까
#### 2. 우리 모듈에서 발생한 예외가 모듈을 호출하는 코드로 아주 멀리 전달되서 프로그램이 깨지는 상황 예방

<br>

### 보호의 유용한 효과 3가지
### 1. API사용자가 API를 잘못 호출한 경우, 더 쉽게 실수를 이해할 수 있다.
API를 제대로 사용하려면 API에서 의도적으로 발생시키는 에러를 잡아내야만 한다.   
만약 사용자가 이것을 안하면, 모듈에 정의한 최상위 예외를 잡아내는 except블록까지 예외가 전달되어   
**API사용자의 주의를 환기시키고, 실수한 예외타입을 제대로 처리할 기회를 준다.**

In [4]:
try:
    weight = my_module.determine_weight(-1, 1)
except my_module.InvalidDensityError:
    weight = 0
except my_module.Error:
    logging.exception('호출 코드에 버그가 있음.')

ERROR:root:호출 코드에 버그가 있음.
Traceback (most recent call last):
  File "<ipython-input-4-b692760df814>", line 2, in <module>
    weight = my_module.determine_weight(-1, 1)
  File "C:\Users\PC\Documents\GitHub\into_the_python1\AIFFEL\Python Better Ways\my_module.py", line 33, in determine_weight
    raise InvalidVolumeError('부피는 0보다 커야 합니다.')
my_module.InvalidVolumeError: 부피는 0보다 커야 합니다.


### 2. API모듈 코드의 버그 발견시 도움이 됨.
1. 우리가 작성한 모듈 코드는 모듈 내에서 정의한 예외 계층에 속하는 예외만 발생시킬 수 있다. 
2. **다른 타입의 에러가 발생했다면** 이것은 우리가 의도하지 않은 것.   
   **= 우리가 구현한 API코드에 버그가 있다!!!**

### 호출하는 쪽에서 Python기반의 Except 클래스를 잡는 다른 except블록 추가해야 한다.
1.의 try/except문이 모듈의 버그로 부터 API소비자를 보호하지 못하므로!

###  즉, 1.과 2.모두 구현하면 API사용자가 모듈에 수정해야 할 버그가 있는 경우를 쉽게 알 수 있다.

In [5]:
try:
    weight = my_module.determine_weight(0,1)
except my_module.InvalidDensityError:
    weight = 0
except my_module.Error:
    logging.exception('호출 코드에 버그가 있음')
except Exception:
    logging.exception('API 코드에 버그가 있음!')
    raise # 예외를 호출자 쪽으로 다시 발생시킴

ERROR:root:API 코드에 버그가 있음!
Traceback (most recent call last):
  File "<ipython-input-5-2352a62bd195>", line 2, in <module>
    weight = my_module.determine_weight(0,1)
  File "C:\Users\PC\Documents\GitHub\into_the_python1\AIFFEL\Python Better Ways\my_module.py", line 35, in determine_weight
    density / volume  # density를 0으로 나누는 것은 저자의 의도적인 버그
ZeroDivisionError: division by zero


ZeroDivisionError: division by zero

### 3. 미래의 API를 보호
#### API를 지속적으로 확장해 특정상황에서 더 구체적인 예외를 제공해야 할 때 유용

In [ ]:
# Ex. 밀도가 음수인 경우를 오류 조건으로 표시해주는 Exception 하위 클래스 추가.

class NegativeDensityError(InvalidDensityError):
    """밀도가 음수인 경우"""
    
def determine_weight(volume, density):
    if density < 0:
        raise NegativeDensityError('밀도는 0보다 커야 합니다')

### 모듈 호출하는 코드 변경없이 예전처럼 잘 작동한다.
InvalidDensityError 예외(NegativeDensityError의 부모 클래스)를 이미 처리하기 때문!

### 나중에 호출하는 코드에서 새로운 타입의 예외를 더 처리해야할 때도 적절히 수정가능

In [6]:
try:
    weight = my_module.determine_weight(1,-1)
except my_module.NegativeDensityError as exc:
    raise ValueError('밀도로 음수가 아닌 값을 제공해야 합니다.') from exc
except my_module.InvalidDensityError:
    weight = 0
except my_module.Error:
    logging.exception('호출 코드에 버그가 있음')
except Exception:
    logging.exception('API 코드에 버그가 있음!')
    raise

ValueError: 밀도로 음수가 아닌 값을 제공해야 합니다.

### 최상위 예외 바로 아래에 폭넓은 예외 상황을 표현하는 다양한 오류 제공시
### 미래의 코드 변경에 대한 보호를 더 강화!
ex) 무게 계산 관련 예외, 부피 계산 관련 예외, 밀도 계산 관련 예외를 추가하는 경우등

In [1]:
class Error(Exception):
    """이 모듈에서 발생할 모든 예외의 상위 클래스."""

class WeightError(Error):
    """부피 계산 관련 예외의 상위 클래스"""

class VolumeError(Error):
    """부피 계산 관련 예외의 상위 클래스"""

class DensityError(Error):
    """밀도 계산 관련 예외의 상위 클래스"""

구체적인 예외는 이런 일반적인 예외를 상속   
각 중간단계 예외는 각각  서로 다른 예외상황에 대해 최상위 예외 역할을 함.

### API코드로부터 API를 호출하는 계층을 쉽게 추가할 수 있다.

### 모든 호출 코드가 구체적인 Exception 하위 클래스 예외를 일일이 처리하게 하는 것보다 낫다.

### 요약
* 모듈에서 사용할 **최상위 예외를 정의하면 API 사용자들이 자신을 API로부터 보호**할 수 있다.
* **최상위 예외**를 잡아내면 **API를 소비하는 코드의 버그**를 쉽게 찾을 수 있다.
* **파이썬Exception 기반 클래스**를 잡아내면 **API 구현의 버그**를 쉽게 찾을 수 있다.
* **중간 단계의 최상위 예외**를 사용하면, 미래에 **새로운 타입의 예외를 API에 추가할 때 API를 사용하는 코드가 깨지는 일을 방지**할 수 있다.